In [1]:
import pickle
import sys
sys.path.append('../src/')
from SpacedRepition import SpacedRepetitionModel as SRM


In [2]:
with open('../cleaned/1015_train_test.pkl', 'rb') as handle:
    train_test = pickle.load(handle)

In [3]:
test = train_test['test']
train = train_test['train']
del train_test

In [4]:
test.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,...,p_recall_clip,delta_days,log_p_recall_clip,half_life,wrong,session_wrong,right_sqrt,wrong_sqrt,bias,a
11568803,1.0,1363009539,2829619,u:dHVF,fr,en,ccf02ea2e390aea793ebf62b0c490075,<*sf>/indispensable<adj><mf><*numb>,2,2,...,0.9999,32.750220,-0.000144,274.0,0,1,1.732051,1.000000,1,0.666667
11568804,1.0,1363009539,1974004,u:dHVF,fr,en,97835f097be96da3ccb1e9b014b482fc,<*sf>/faux<adj><*gndr><*numb>,3,3,...,0.9999,22.847269,-0.000144,274.0,0,1,2.000000,1.000000,1,0.714286
11568805,1.0,1363009539,1974004,u:dHVF,fr,en,9c5f150b7ade77c73c8042af28b65ac0,<*sf>/tel<adj><*gndr><*numb>,2,1,...,0.9999,22.847269,-0.000144,274.0,1,1,1.414214,1.414214,1,0.500000
11568806,1.0,1363009539,1974004,u:dHVF,fr,en,1932e8abc8d82e23f343cdc4d84b8bbe,<*sf>/juste<adj><mf><*numb>,1,1,...,0.9999,22.847269,-0.000144,274.0,0,1,1.414214,1.000000,1,0.600000
11568807,1.0,1363009539,2156799,u:dHVF,fr,en,1f143f9a64cb3dd127ffc89c71840f55,<*sf>/fin<adj><*gndr><*numb>,1,1,...,0.9999,24.962951,-0.000144,274.0,0,1,1.414214,1.000000,1,0.600000


In [5]:
train.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,...,p_recall_clip,delta_days,log_p_recall_clip,half_life,wrong,session_wrong,right_sqrt,wrong_sqrt,bias,a
0,1.0,1362076081,27649635,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,lernt/lernen<vblex><pri><p3><sg>,6,4,...,0.9999,320.018924,-0.000144,274.0,2,2,2.236068,1.732051,1,0.600000
1,0.5,1362076081,27649635,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,die/die<det><def><f><sg><nom>,4,4,...,0.5000,320.018924,-1.000000,274.0,0,1,2.236068,1.000000,1,0.750000
2,1.0,1362076081,27649635,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,mann/mann<n><m><sg><nom>,5,4,...,0.9999,320.018924,-0.000144,274.0,1,1,2.236068,1.414214,1,0.666667
3,0.5,1362076081,27649635,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,frau/frau<n><f><sg><nom>,6,5,...,0.5000,320.018924,-1.000000,274.0,1,1,2.449490,1.414214,1,0.700000
4,1.0,1362076081,27649635,u:FO,de,en,84920990d78044db53c1b012f5bf9ab5,das/das<det><def><nt><sg><nom>,4,4,...,0.9999,320.018924,-0.000144,274.0,0,1,2.236068,1.000000,1,0.750000


In [6]:
print(f"train shape is {train.shape}")
print(f"test shape is {test.shape}")

train shape is (11568803, 22)
test shape is (1285423, 22)
